In [ ]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview
All the code we'll look at is in the next cell. We will step through each step after.

In [ ]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

from tensorflow.contrib.learn.python.learn.datasets import base

# Data files
IRIS_TRAINING = "data/iris_training.csv"
IRIS_TEST = "data/iris_test.csv"

# Load datasets.
training_set = base.load_csv_with_header(filename=IRIS_TRAINING,
                                         features_dtype=np.float32,
                                         target_dtype=np.int)
test_set = base.load_csv_with_header(filename=IRIS_TEST,
                                     features_dtype=np.float32,
                                     target_dtype=np.int)

# Specify that all features have real-value data
feature_name = "flower_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, 
                                                    shape=[4])]
classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="/tmp/iris_model")

def input_fn(dataset):
    def _fn():
        features = {feature_name: tf.constant(dataset.data)}
        label = tf.constant(dataset.target)
        return features, label
    return _fn

# Fit model.
classifier.train(input_fn=input_fn(training_set),
               steps=1000)
print('fit done')

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=100)["accuracy"]
print('\nAccuracy: {0:f}'.format(accuracy_score))

# Export the model for serving
feature_spec = {'flower_features': tf.FixedLenFeature(shape=[4], dtype=np.float32)}

serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir_base='/tmp/iris_model' + '/export', 
                            serving_input_receiver_fn=serving_fn)


## Imports

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

/root/anaconda3/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


## Data set
From https://en.wikipedia.org/wiki/Iris_flower_data_set

3 types of Iris Flowers: 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/450px-Kosaciec_szczecinkowaty_Iris_setosa.jpg" style="width: 100px; display:inline"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/800px-Iris_versicolor_3.jpg" style="width: 150px;display:inline"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/736px-Iris_virginica.jpg" style="width: 150px;display:inline"/>
* Iris Setosa
* Iris Versicolour
* Iris Virginica




## Data Columns:
   1. sepal length in cm 
   2. sepal width in cm 
   3. petal length in cm 
   4. petal width in cm

<img src="petal_sepal.png" style="width: 200px;"/>
<img src="https://storage.googleapis.com/image-uploader/AIA_images/data_table.png" style="width: 450px"/>

## Load data in

In [3]:
from tensorflow.contrib.learn.python.learn.datasets import base

# Data files
IRIS_TRAINING = "data/iris_training.csv"
IRIS_TEST = "data/iris_test.csv"

# Load datasets.
training_set = base.load_csv_with_header(filename=IRIS_TRAINING,
                                         features_dtype=np.float32,
                                         target_dtype=np.int)
test_set = base.load_csv_with_header(filename=IRIS_TEST,
                                     features_dtype=np.float32,
                                     target_dtype=np.int)

print(training_set.data)

print(training_set.target)

[[ 6.4000001   2.79999995  5.5999999   2.20000005]
 [ 5.          2.29999995  3.29999995  1.        ]
 [ 4.9000001   2.5         4.5         1.70000005]
 [ 4.9000001   3.0999999   1.5         0.1       ]
 [ 5.69999981  3.79999995  1.70000005  0.30000001]
 [ 4.4000001   3.20000005  1.29999995  0.2       ]
 [ 5.4000001   3.4000001   1.5         0.40000001]
 [ 6.9000001   3.0999999   5.0999999   2.29999995]
 [ 6.69999981  3.0999999   4.4000001   1.39999998]
 [ 5.0999999   3.70000005  1.5         0.40000001]
 [ 5.19999981  2.70000005  3.9000001   1.39999998]
 [ 6.9000001   3.0999999   4.9000001   1.5       ]
 [ 5.80000019  4.          1.20000005  0.2       ]
 [ 5.4000001   3.9000001   1.70000005  0.40000001]
 [ 7.69999981  3.79999995  6.69999981  2.20000005]
 [ 6.30000019  3.29999995  4.69999981  1.60000002]
 [ 6.80000019  3.20000005  5.9000001   2.29999995]
 [ 7.5999999   3.          6.5999999   2.0999999 ]
 [ 6.4000001   3.20000005  5.30000019  2.29999995]
 [ 5.69999981  4.4000001   1.5 

## Feature columns and model creation

In [4]:
# Specify that all features have real-value data
feature_name = "flower_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, 
                                                    shape=[4])]

classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="/tmp/iris_model")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f20e897ec88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
feature_columns

[_NumericColumn(key='flower_features', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Input function

In [6]:
def input_fn(dataset):
    def _fn():
        features = {feature_name: tf.constant(dataset.data)}
        label = tf.constant(dataset.target)
        return features, label
    return _fn

print(input_fn(training_set)())

# raw data -> input function -> feature columns -> model

({'flower_features': <tf.Tensor 'Const:0' shape=(120, 4) dtype=float32>}, <tf.Tensor 'Const_1:0' shape=(120,) dtype=int64>)


## Training

In [7]:
# Fit model.
classifier.train(input_fn=input_fn(training_set),
               steps=1000)
print('fit done')


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 131.833, step = 1
INFO:tensorflow:global_step/sec: 1429.15
INFO:tensorflow:loss = 37.1391, step = 101 (0.071 sec)
INFO:tensorflow:global_step/sec: 1955.61
INFO:tensorflow:loss = 27.8594, step = 201 (0.051 sec)
INFO:tensorflow:global_step/sec: 1981.7
INFO:tensorflow:loss = 23.0449, step = 301 (0.051 sec)
INFO:tensorflow:global_step/sec: 1991.38
INFO:tensorflow:loss = 20.058, step = 401 (0.050 sec)
INFO:tensorflow:global_step/sec: 1929.66
INFO:tensorflow:loss = 18.0083, step = 501 (0.052 sec)
INFO:tensorflow:global_step/sec: 2112.27
INFO:tensorflow:loss = 16.505, step = 601 (0.047 sec)
INFO:tensorflow:global_step/sec: 1894.94
INFO:tensorflow:loss = 15.3496, step = 701 (0.053 sec)
INFO:tensorflow:global_step/sec: 1978.08
INFO:tensorflow:loss = 14.43, step = 801 (0.051 sec)
INFO:tensorflow:global_step/sec: 1916.46
INFO:tensorflow:loss = 13.6782, step 

In [6]:
## Evaluation

In [15]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=input_fn(test_set), steps=100)["accuracy"]
print('\nAccuracy: {0:f}'.format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2018-06-13-11:38:06
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1000
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorf

In [16]:
classifier.evaluate?

# Estimators review

### Load datasets.

    training_data = load_csv_with_header()

### define input functions

    def input_fn(dataset)
   
### Define feature columns

    feature_columns = [tf.feature_column.numeric_column(feature_name, shape=[4])]

### Create model

    classifier = tf.estimator.LinearClassifier()

### Train

    classifier.train()

### Evaluate

    classifier.evaluate()

## Exporting a model for serving predictions


In [17]:
feature_spec = {'flower_features': tf.FixedLenFeature(shape=[4], dtype=np.float32)}

serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir_base='/tmp/iris_model' + '/export', 
                            serving_input_receiver_fn=serving_fn)





INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"/tmp/iris_model/export/temp-b'1528889978'/saved_model.pb"


b'/tmp/iris_model/export/1528889978'